# Relax Take Home Challenge - Analysis

Some quick EDA showed that the data in the form given to us didn't provide many distinguishing features, so I decided to create some new features.  Common sense suggests that those who login shortly after generating their accounts and those who login more frequently are most likely to become adopted users.

## Cleaning and Feature Engineering Steps
### 1. identify adopted users
To identify the adopted users (users who logged in at least three times over a seven day period), I added a column to count the number of visits over a rolling seven day period:

```python
def get_rolling_count(grp, freq):
    return grp.rolling(freq, on='time_stamp')['user_id'].count()

engage['visits_7_days'] = engage.groupby('user_id', as_index=False, group_keys=False).apply(get_rolling_count, '7D')
```

Then, I collected each row >= 3, dropped duplicates after the first occurrence and converted the results to a list. I used `.isin` to check whether the `object_id` in the users DataFrame was in the list of adopted users and marked matches as `True` in the users DataFrame.


### 2. add col for time between account creation and initial login
I grouped by `user_id` and used `np.min` with a `.agg` function to find the first login for each user, added that time stamp to the users DataFrame and created a new column to record the time in days between creation of each account and first login.

### 3. add col for mean time between logins
I created a new DataFrame to calculate and store the time between logins:

```python
gap = engage.groupby('user_id').time_stamp.apply(lambda x: x - x.shift()).dt.days
```

I then grouped these by user, calculated a mean time between all logins for each user and merged the resulting Series into the users DataFrame.

### 4. add col - org size categorical

I used `.value_counts()` to generate a dictionary counting the number of users who belonged to each org, then binned these into XS, S, M, L and XL and each these to users in the users DataFrame:

```python
def org_size(ct):
    rtn = 'XS'
    if ct > 200:
        rtn = 'XL'
    elif ct > 149:
        rtn = 'L'
    elif ct > 99:
        rtn = 'M'
    elif ct > 49:
        rtn = 'S'

    return rtn

o['org_size'] = o.user_count.apply(org_size)

org_size_dict = dict(zip(o.org_id, o.org_size))
org_size_dict[0] = np.nan

users['org_size'] = users.org_id.map(org_size_dict)
```

### 5. add col - invited by user group size

I used a strategy similar to the org size categorizing and binning described above to make each user's social network within the app into a feature binned by size.

### 6. drop unneeded cols
I dropped unneeded columns, such as `object_id`, `name`, `email`, etc.

### 7. transform categoricals
I used `pd.get_dummies` to one-hot encode `creation_size`, `org_size` and `group_size`.


## Modeling

The target set was imbalanced, with approximately 13% of all users labeled positively, as adopted users. While not massively imbalanced, I decided to use the SMOTE technique from `imblearn` to amplify the signal in the noise.  Because the goal of the project is "to identify which factors predict future user adoption", I chose to focus on precision scores and the models that produced the highest number of true positives.  I used sklearn's dummy classifier as a baseline, which produced the following classification report:

![](images/dummy_classifier.png)

I ran the dataset through Logistic Regression, LinearSVC, SVC and Random Forest.  The results from Random Forest were the best, so I tuned the hyperparameters and re-ran the model with tuned parameters.  The tuned model results in a slightly higher F1 score, but the precision score was a mixed bag -- the number of true positives increased from 411 to 472, but the number of false positives also increased from 59 to 129.  Below are the results of the tuned Random Forest:

![](images/tuned_rf.png)



The feature importances aren't surprising.  `mean_gap_length` indicates the mean gap between logins in days.  Since this is so closely related to the definition of an adopted user, it's not surprising that the two are highly correlated. This is still a useful bit of information.  The company could send tickler emails or find other ways to encourage users login, particularly within the days immediately following account creation.  The second most important feature, by a wide margin, is the gap in days between account creation and initial login.